## Assignment 3: Transportation problem

In the transportation problem our goal is to minimize the cost of transporting a product from a given number of suppliers to a given number of destinations. The problem is solved by Linear Programming.

- There are $I$ suppliers $F_i$, and the possible supply of each supplier is denoted by $S_i$
<br><br>
- The demand of each destination warehouse $W_j$ is denoted by $D_j$
<br><br>
- The per-unit cost of transport from a supplier to destination is given by $c_{ij}$
<br><br>
- The quantity of product / material that is transported between supplier $i$ and destination $j$ is given by $x_{ij}$

### Task 1: Formulate the transportation problem as an LP

Given the above definitions we can formulate the problem as follows

$$
    \textit{min} \; \sum_{i=1}^{M}\sum_{j=1}^{N}c_{ij}x_{ij}
$$

Subject to the constraints:

$$ \sum_{j=1}^{N}x_{ij} \leq S_i, i = 1,...., M $$

$$ \sum_{i=1}^{M}x_{ij} \leq D_j, j = 1,...., N $$

$$ x_{ij}\geq 0, i = 1,...., M, j = 1,...., N $$

In standard form this becomes

$$
    \textit{min} \quad c^Tx
$$
$$
    \textit{s.t.} \quad Ax \leq b
$$
$$
    \qquad x \geq 0
$$
<br><br>
In other words, we cannot ship more than the smallest of total supply and demand. There are different possible cases, e.g. 
- $S = D$: supply matches demand; then we get equalitities in the sums above (task 2)
<br><br>
- $S < D$: there is more demand than what we can produce
<br><br>
- $S > D$: there is a surplus (this is the case in task 4)

### Task 2: Solve the problem using a python solver

We are given the following data and we use the `linprog` solver from `scipy.optimize` to minimize the transportation costs subject to the constraints.

<img src="transport-table.png">

In this first case we see that $S = D = 115$.

[**TODO: Add what the different matrices are in markdown**]

In [2]:
import numpy as np
from scipy.optimize import linprog

In [3]:
# Set up vectors and matrices required for the optimization

# Number of warehouses and factories
# M = 3
# N = 4

# Supply from each factory F
S = np.array([25, 55, 35])

# Demand in each warehouse W
D = np.array([15, 45, 30, 25])

# Transport cost grid
c = np.array([10, 0, 20, 11, 
              12, 7, 9, 20, 
              0, 14, 16, 18])

# Constraint coefficient matrix
A = np.array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
              [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
              [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]])

# Putting all constraint values into one vector
b = np.array([25, 55, 35, 15, 45, 30, 25])

In this first case, the constraints are formulated as equalities. The `scipy.linprog` "HiGHS" picks automatically between simplex and interior point methods, depending on the problem.

In [4]:
# Run solver with equalities

optim = linprog(c, A_eq=A , b_eq=b, method="highs")

In [5]:
optim

           con: array([0., 0., 0., 0., 0., 0., 0.])
 crossover_nit: 0
         eqlin:  marginals: array([-0.,  7.,  7., -7., -0.,  2., 11.])
  residual: array([0., 0., 0., 0., 0., 0., 0.])
           fun: 860.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: <MemoryView of 'ndarray' at 0x7fa040e2d1e0>
  residual: array([ 0., 20.,  0.,  5.,  0., 25., 30.,  0., 15.,  0.,  0., 20.])
       message: 'Optimization terminated successfully.'
           nit: 6
         slack: array([], dtype=float64)
        status: 0
       success: True
         upper:  marginals: <MemoryView of 'ndarray' at 0x7fa040e2d040>
  residual: array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf])
             x: array([ 0., 20.,  0.,  5.,  0., 25., 30.,  0., 15.,  0.,  0., 20.])

**Result:** The result of the LP is given in the table below, where each cell contains the quantities $x_ij$ between factory $F_i$ and warehouse $W_j$.

<img src="q_task_2.png">

### Task 3: Compare simplex and interior point methods

In this task, we specify which method to use and compare the results. Both the simplex and interior point methods give the same optimal allocations. The also take the same number of iterations.

In [6]:
# Run using simplex algorithm
simplex = linprog(c, A_eq=A , b_eq=b, method="highs-ds")

# Run using interior point algorithm
int_point = linprog(c, A_eq=A , b_eq=b, method="highs-ipm")

In [7]:
simplex

           con: array([0., 0., 0., 0., 0., 0., 0.])
 crossover_nit: 0
         eqlin:  marginals: array([-0.,  7.,  7., -7., -0.,  2., 11.])
  residual: array([0., 0., 0., 0., 0., 0., 0.])
           fun: 860.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: <MemoryView of 'ndarray' at 0x7fa040e2dd40>
  residual: array([ 0., 20.,  0.,  5.,  0., 25., 30.,  0., 15.,  0.,  0., 20.])
       message: 'Optimization terminated successfully.'
           nit: 6
         slack: array([], dtype=float64)
        status: 0
       success: True
         upper:  marginals: <MemoryView of 'ndarray' at 0x7fa040e2dba0>
  residual: array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf])
             x: array([ 0., 20.,  0.,  5.,  0., 25., 30.,  0., 15.,  0.,  0., 20.])

In [8]:
int_point

           con: array([0., 0., 0., 0., 0., 0., 0.])
 crossover_nit: 0
         eqlin:  marginals: array([ 11.,  18.,  18., -18., -11.,  -9.,  -0.])
  residual: array([0., 0., 0., 0., 0., 0., 0.])
           fun: 860.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: <MemoryView of 'ndarray' at 0x7f9ff000c1e0>
  residual: array([ 0., 20.,  0.,  5.,  0., 25., 30.,  0., 15.,  0.,  0., 20.])
       message: 'Optimization terminated successfully.'
           nit: 6
         slack: array([], dtype=float64)
        status: 0
       success: True
         upper:  marginals: <MemoryView of 'ndarray' at 0x7f9ff000c040>
  residual: array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf])
             x: array([ 0., 20.,  0.,  5.,  0., 25., 30.,  0., 15.,  0.,  0., 20.])

### Task 4: Surplus supply leads to supply-demand imbalance

Now $S_2$ increases from $55$ to $60$, which means that $S > D$. We therefore have an optimization problem with an inequality for supply. To solve this we introduce a 5th "artificial" warehouse with $5$ in demand which means that we can keep equality in the solver itself.

<img src="task_4_input.png">

In [22]:
# Adding a 0 entry for each row in the original matrix
c_2 = np.array([10, 0, 20, 11, 0, 12, 7, 9, 20, 0, 0, 14, 16, 18, 0])

# Changing second supply entry to 60 and adding one more demand entry of 5
b_2 = np.array([25, 60, 35, 15, 45, 30, 25, 5])

# Making corresponding adjustments to A
A_2 = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
                [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

In [32]:
# Run using simplex algorithm
simplex_2 = linprog(c_2, A_eq=A_2 , b_eq=b_2, method="highs-ds")

# Run using interior point algorithm
int_point_2 = linprog(c_2, A_eq=A_2 , b_eq=b_2, method="highs-ipm")

In [33]:
simplex_2

           con: array([0., 0., 0., 0., 0., 0., 0., 0.])
 crossover_nit: 0
         eqlin:  marginals: array([-7., -0., -0., -0.,  7.,  9., 18., -0.])
  residual: array([0., 0., 0., 0., 0., 0., 0., 0.])
           fun: 860.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: <MemoryView of 'ndarray' at 0x7f9ff002f040>
  residual: array([ 0., 15.,  0., 10.,  0.,  0., 30., 30.,  0.,  0., 15.,  0.,  0.,
       15.,  5.])
       message: 'Optimization terminated successfully.'
           nit: 8
         slack: array([], dtype=float64)
        status: 0
       success: True
         upper:  marginals: <MemoryView of 'ndarray' at 0x7f9ff001fd40>
  residual: array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf])
             x: array([ 0., 15.,  0., 10.,  0.,  0., 30., 30.,  0.,  0., 15.,  0.,  0.,
       15.,  5.])

In [34]:
int_point_2

           con: array([0., 0., 0., 0., 0., 0., 0., 0.])
 crossover_nit: 1
         eqlin:  marginals: array([-7.,  0.,  0.,  0.,  7.,  9., 18., -0.])
  residual: array([0., 0., 0., 0., 0., 0., 0., 0.])
           fun: 860.0
       ineqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
         lower:  marginals: <MemoryView of 'ndarray' at 0x7f9ff002f380>
  residual: array([ 0., 20.,  0.,  5.,  0.,  0., 25., 30.,  0.,  5., 15.,  0.,  0.,
       20.,  0.])
       message: 'Optimization terminated successfully.'
           nit: 5
         slack: array([], dtype=float64)
        status: 0
       success: True
         upper:  marginals: <MemoryView of 'ndarray' at 0x7f9ff002f1e0>
  residual: array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf])
             x: array([ 0., 20.,  0.,  5.,  0.,  0., 25., 30.,  0.,  5., 15.,  0.,  0.,
       20.,  0.])

**Result:** The updated quantities are given below, with increases in green and decreases in yellow

<img src="q_task_4.png">